In [1]:

import DatabaseApi
import pandas as pd


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:

def getTeams():
        teams = DatabaseApi.getList('Tm','Players')[0]
        mod_teams=[]
        for team in teams:
            mod_teams.append(team[0])
        return mod_teams
    
def populatePlayers():
    for player_key in data_dict['player']:
        
        teams = getTeams()
        
        if player_key in teams:
            
            players= DatabaseApi.getStats('Player','Players','Tm',player_key)[0]
            for p in players:
                data_dict["player"].append(p[-1])
    
    data_dict['player'] = [val for val in data_dict['player'] if val not in teams]
    
def getAvgStats(gt="G"):
    ret=[]
    player_data_dict={}
    result={}
    for player_key in data_dict['player']:

        #db_data = DatabaseApi.getStats('*','Players','player',player_key)
        select_v = ', '.join(['AVG(\"{}\")'.format(item) if item[0].isdigit() else 'AVG({})'.format(item) for item in data_dict['stats']])
        db_data  = DatabaseApi.getBoxScoreStats(select_v,'BoxScores',['Player','BoxScore_Type'], [player_key,gt],None,'AVG') 
        if db_data == 0:
            continue
        player_data_dict[player_key] = dict(zip(db_data[1],db_data[0][0]))
        result[player_key]  = player_data_dict[player_key]
        #result[player_key] = { key: player_data_dict[player_key][key] for key in data_dict['stats'] if key in player_data_dict[player_key]}
        ret.append([player_key,result[player_key]])
        result={}
    return ret

def getXGames(x):
    ret=[]
    result={}
  
    player_data_dict={}
    for player_key in data_dict["player"]:
        
        for gt in data_dict['filter']['GameType']:
            special = {'AVG':{},'SUM':{}}
            
            for i in data_dict['stats']:
                if '(' in i and i[:i.index('(')] in special.keys():
                    special[i[:i.index('(')]][i[i.index('(')+1:i.index(')')]]=0
                    data_dict['stats'][data_dict['stats'].index(i)]= i[i.index('(')+1:i.index(')')]
                
            select_v = ', '.join(['\"{}\"'.format(item) if item[0].isdigit() else '{}'.format(item) for item in data_dict['stats']])
            select_v += ', Opponent'
            num = int(x[1:])-1
            db_data  = DatabaseApi.getBoxScoreStats(select_v,'BoxScores',['Player','BoxScore_Type'], [player_key,gt],num,'DESC' if x[0]=='L' else 'ASC') 
            if db_data == 0:
                continue
            if num >len(db_data[0])-1:
                num = len(db_data[0])-1
            i=0
            av = 0
            while i<=num:
                
                player_data_dict[player_key] = dict(zip(db_data[1],db_data[0][i]))
                result[player_key+" VS "+player_data_dict[player_key]["Opponent"]+" "+x[0]+str(i+1)] = { key: player_data_dict[player_key][key] for key in data_dict['stats'] if key in player_data_dict[player_key]}
                if db_data[0][i][0] == None:
                    i=i+1
                    av+=1
                    continue
                for st in db_data[1]:
                    for y in special:
                        if st in special[y]:
                            special[y][st]+=player_data_dict[player_key][st]              

                i=i+1
            
            
            for st in special['AVG']:
                    if (int(x[1:]) - av) != 0:
                        special["AVG"][st]/= int(x[1:]) - av
        
        ret.append([player_key,special["AVG"]])
            
    return ret

In [3]:

data_dict = {'player': [], 'team': [], 'stats': [], 'schedule':[],
                  
                  'filter':{ 
                      'GameType':[],
                      'XGames':[],
                      'DateRange':[],
                      'Date':[],
                      'PlayerOrTeam':[]
                  }, 
                  
                  }
special={}
data_dict["player"] = getTeams()
populatePlayers()
players=data_dict["player"]
mapping = []
for player in players:
    data_dict["player"] = [player]
    data_dict["stats"] = ['PTS+TRB+AST']
    data_dict["filter"]["GameType"] = ['G']
    t=getAvgStats()
    if t!=[]:
        mapping.append(getAvgStats()[0]) 
    

data_dict = {'player': [], 'team': [], 'stats': [], 'schedule':[],
                  
                  'filter':{ 
                      'GameType':[],
                      'XGames':[],
                      'DateRange':[],
                      'Date':[],
                      'PlayerOrTeam':[]
                  }, 
                  
                  }
special={}
data_dict["player"] = getTeams()
populatePlayers()

players=data_dict["player"]
for player in players:
    data_dict["player"] = [player]
    data_dict["stats"] = ['AVG(PTS+TRB+AST)']
    data_dict["filter"]["XGames"] =['L3']
    data_dict["filter"]["GameType"] = ['G']
    
    if t!=[]:
        mapping.append(getXGames("L3")[0])
compare= {}

for item in mapping:
    if item[0] in compare:
        compare[item[0]].update(item[1])
    else:
        compare[item[0]] = item[1]
df = pd.DataFrame.from_dict(compare).T
df["DIFF"] = df["PTS+TRB+AST"]-(df["AVG(PTS+TRB+AST)"])
df = df.sort_values(by=["DIFF"],ascending=False)
df.head(20)
del df["AVG(PTS+TRB+AST)"]
del df["PTS+TRB+AST"]
del df["DIFF"]
x=df.head(40)
str(x)




'Empty DataFrame\nColumns: []\nIndex: [Dalano Banton, Josh Giddey, Sandro Mamukelashvili, Miles McBride, Amen Thompson, Bogdan Bogdanović, RJ Barrett, Charles Bassey, Jalen Green, Cam Thomas, Victor Wembanyama, Marcus Smart, Rui Hachimura, Dejounte Murray, Vasilije Micić, Haywood Highsmith, Marcus Sasser, Jalen Brunson, Simone Fontecchio, Trendon Watford, Grant Williams, Jordan Poole, Javon Freeman-Liberty, Nate Hinton, Keon Ellis, Julian Champagnie, Trayce Jackson-Davis, Kelly Olynyk, Toumani Camara, Devin Vassell, Giannis Antetokounmpo, Nicolas Batum, Anthony Gill, Rayan Rupert, Mo Bamba, Josh Hart, Collin Sexton, John Collins, Malachi Flynn, Deni Avdija]'

In [4]:


data_dict = {'player': [], 'team': [], 'stats': [], 'schedule':[],
                  
                  'filter':{ 
                      'GameType':[],
                      'XGames':[],
                      'DateRange':[],
                      'Date':[],
                      'PlayerOrTeam':[]
                  }, 
                  
                  }
special={}
data_dict["player"] = getTeams()
populatePlayers()
players=data_dict["player"]
mapping = []
for player in players:
    data_dict["player"] = [player]
    data_dict["stats"] = ['PTS']
    data_dict["filter"]["GameType"] = ['G']
    t=getAvgStats()
    if t!=[]:
        mapping.append(getAvgStats()[0]) 
    

data_dict = {'player': [], 'team': [], 'stats': [], 'schedule':[],
                  
                  'filter':{ 
                      'GameType':[],
                      'XGames':[],
                      'DateRange':[],
                      'Date':[],
                      'PlayerOrTeam':[]
                  }, 
                  
                  }
special={}
data_dict["player"] = getTeams()
populatePlayers()

players=data_dict["player"]
for player in players:
    data_dict["player"] = [player]
    data_dict["stats"] = ['AVG(PTS)']
    data_dict["filter"]["XGames"] =['L3']
    data_dict["filter"]["GameType"] = ['G']
    
    if t!=[]:
        mapping.append(getXGames("L3")[0])
compare= {}

for item in mapping:
    if item[0] in compare:
        compare[item[0]].update(item[1])
    else:
        compare[item[0]] = item[1]
df = pd.DataFrame.from_dict(compare).T
df["DIFF"] = df["PTS"]-(df["AVG(PTS)"])
df = df.sort_values(by=["DIFF"],ascending=False)
df.head(20)
del df["AVG(PTS)"]
del df["PTS"]
del df["DIFF"]
x=df.head(40)
str(x)


    

'Empty DataFrame\nColumns: []\nIndex: [Dalano Banton, Marcus Smart, Jalen Brunson, Josh Giddey, Miles McBride, Cade Cunningham, Cam Thomas, Victor Wembanyama, Bogdan Bogdanović, Haywood Highsmith, Rui Hachimura, Dejounte Murray, Simone Fontecchio, Marcus Sasser, John Collins, Amen Thompson, Jalen Green, Toumani Camara, Javon Freeman-Liberty, Devin Vassell, Sandro Mamukelashvili, Nate Hinton, Collin Sexton, Anthony Gill, Dennis Schröder, Trendon Watford, Gradey Dick, Nicolas Batum, Julian Champagnie, RJ Barrett, Gary Trent Jr., Jarred Vanderbilt, Garrison Mathews, Malachi Flynn, Jabari Smith Jr., Keon Ellis, Lamar Stevens, Mike Conley, Cory Joseph, Patty Mills]'

In [5]:

data_dict = {'player': [], 'team': [], 'stats': [], 'schedule':[],
                  
                  'filter':{ 
                      'GameType':[],
                      'XGames':[],
                      'DateRange':[],
                      'Date':[],
                      'PlayerOrTeam':[]
                  }, 
                  
                  }
special={}
data_dict["player"] = getTeams()
populatePlayers()
players=data_dict["player"]
mapping = []
for player in players:
    data_dict["player"] = [player]
    data_dict["stats"] = ['AST']
    data_dict["filter"]["GameType"] = ['G']
    t=getAvgStats()
    if t!=[]:
        mapping.append(getAvgStats()[0]) 
    

data_dict = {'player': [], 'team': [], 'stats': [], 'schedule':[],
                  
                  'filter':{ 
                      'GameType':[],
                      'XGames':[],
                      'DateRange':[],
                      'Date':[],
                      'PlayerOrTeam':[]
                  }, 
                  
                  }
special={}
data_dict["player"] = getTeams()
populatePlayers()

players=data_dict["player"]
for player in players:
    data_dict["player"] = [player]
    data_dict["stats"] = ['AVG(AST)']
    data_dict["filter"]["XGames"] =['L3']
    data_dict["filter"]["GameType"] = ['G']
    
    if t!=[]:
        mapping.append(getXGames("L3")[0])
compare= {}

for item in mapping:
    if item[0] in compare:
        compare[item[0]].update(item[1])
    else:
        compare[item[0]] = item[1]
df = pd.DataFrame.from_dict(compare).T
df["DIFF"] = df["AST"]-(df["AVG(AST)"])
df = df.sort_values(by=["DIFF"],ascending=False)
df.head(20)
del df["AVG(AST)"]
del df["AST"]
del df["DIFF"]
x=df.head(40)
str(x)






"Empty DataFrame\nColumns: []\nIndex: [Vasilije Micić, RJ Barrett, Kelly Olynyk, Killian Hayes, Josh Hart, Darius Garland, Josh Giddey, Luke Kennard, Dalano Banton, Tyrese Maxey, Jordan Poole, Immanuel Quickley, Miles McBride, Cam Thomas, Grant Williams, Pat Connaughton, Devin Vassell, Jalen Green, Jalen Williams, Alex Caruso, Victor Wembanyama, Dejounte Murray, Svi Mykhailiuk, Stephen Curry, Bam Adebayo, Marcus Sasser, D'Moi Hodge, Deni Avdija, Desmond Bane, Austin Reaves, Naz Reid, Tre Mann, Alperen Sengun, Charles Bassey, Jarace Walker, Kyle Kuzma, Gary Trent Jr., Rayan Rupert, Moritz Wagner, Nikola Jović]"

In [6]:

data_dict = {'player': [], 'team': [], 'stats': [], 'schedule':[],
                  
                  'filter':{ 
                      'GameType':[],
                      'XGames':[],
                      'DateRange':[],
                      'Date':[],
                      'PlayerOrTeam':[]
                  }, 
                  
                  }
special={}
data_dict["player"] = getTeams()
populatePlayers()
players=data_dict["player"]
mapping = []
for player in players:
    data_dict["player"] = [player]
    data_dict["stats"] = ['TRB']
    data_dict["filter"]["GameType"] = ['G']
    t=getAvgStats()
    if t!=[]:
        mapping.append(getAvgStats()[0]) 
    

data_dict = {'player': [], 'team': [], 'stats': [], 'schedule':[],
                  
                  'filter':{ 
                      'GameType':[],
                      'XGames':[],
                      'DateRange':[],
                      'Date':[],
                      'PlayerOrTeam':[]
                  }, 
                  
                  }
special={}
data_dict["player"] = getTeams()
populatePlayers()

players=data_dict["player"]
for player in players:
    data_dict["player"] = [player]
    data_dict["stats"] = ['AVG(TRB)']
    data_dict["filter"]["XGames"] =['L3']
    data_dict["filter"]["GameType"] = ['G']
    
    if t!=[]:
        mapping.append(getXGames("L3")[0])
compare= {}

for item in mapping:
    if item[0] in compare:
        compare[item[0]].update(item[1])
    else:
        compare[item[0]] = item[1]
df = pd.DataFrame.from_dict(compare).T
df["DIFF"] = df["TRB"]-(df["AVG(TRB)"])
df = df.sort_values(by=["DIFF"],ascending=False)
df.head(20)
del df["AVG(TRB)"]
del df["TRB"]
del df["DIFF"]
x=df.head(40)
str(x)

"Empty DataFrame\nColumns: []\nIndex: [Sandro Mamukelashvili, Charles Bassey, Thaddeus Young, Amen Thompson, Giannis Antetokounmpo, Dalano Banton, Trayce Jackson-Davis, Anthony Davis, Richaun Holmes, Malcolm Brogdon, Keon Ellis, Trey Murphy III, Kris Dunn, Deni Avdija, Bogdan Bogdanović, Simone Fontecchio, Jalen Green, Mo Bamba, Luka Šamanić, LeBron James, Austin Reaves, KJ Martin, De'Andre Hunter, Jordan Goodwin, P.J. Washington, Rayan Rupert, Josh Hart, Marvin Bagley III, Josh Giddey, John Konchar, Trendon Watford, Rui Hachimura, Grant Williams, Jeremy Sochan, Haywood Highsmith, Santi Aldama, Javon Freeman-Liberty, Kelly Oubre Jr., Thomas Bryant, Aleksej Pokusevski]"